In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielpilao1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielpilao1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gabrielpilao1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
engine = create_engine('sqlite:///../data/DisasterResponse.db')

In [5]:
df = pd.read_sql_table('mess', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
class_count = {}
for index, col in enumerate(df.columns):
    if index > 3:
        class_count[col] = df.iloc[:, index].sum()
    else:
        continue

print(class_count)

{'related': 20450, 'request': 4513, 'offer': 121, 'aid_related': 10954, 'medical_help': 2099, 'medical_products': 1319, 'search_and_rescue': 725, 'security': 471, 'military': 863, 'child_alone': 0, 'water': 1684, 'food': 2956, 'shelter': 2342, 'clothing': 410, 'money': 606, 'missing_people': 301, 'refugees': 880, 'death': 1204, 'other_aid': 3464, 'infrastructure_related': 1709, 'transport': 1212, 'buildings': 1345, 'electricity': 538, 'tools': 159, 'hospitals': 283, 'shops': 120, 'aid_centers': 309, 'other_infrastructure': 1155, 'weather_related': 7343, 'floods': 2177, 'storm': 2464, 'fire': 282, 'earthquake': 2461, 'cold': 533, 'other_weather': 1379, 'direct_report': 5113}


In [34]:
classes = []
number = []

for index, col in enumerate(df.columns):
    if index > 3:
        classes.append(col)
        number.append(df.iloc[:, index].sum())
    else:
        continue

number

[20450,
 4513,
 121,
 10954,
 2099,
 1319,
 725,
 471,
 863,
 0,
 1684,
 2956,
 2342,
 410,
 606,
 301,
 880,
 1204,
 3464,
 1709,
 1212,
 1345,
 538,
 159,
 283,
 120,
 309,
 1155,
 7343,
 2177,
 2464,
 282,
 2461,
 533,
 1379,
 5113]

In [35]:
classes

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [23]:
class_count = {}
for index in range(df.shape[1]):
    if index > 3:
        
        print('Column Number : ', index)
        print(df.iloc[:, index].sum())
    else:
        continue

20450
4513
121
10954
2099
1319
725
471
863
0
1684
2956
2342
410
606
301
880
1204
3464
1709
1212
1345
538
159
283
120
309
1155
7343
2177
2464
282
2461
533
1379
5113


In [6]:
df['genre'].value_counts()

news      13128
direct    10852
social     2406
Name: genre, dtype: int64

In [7]:
Y = df.iloc[:,4:]

In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
Y.sum(axis=0)

related                   20246
request                    4513
offer                       121
aid_related               10954
medical_help               2099
medical_products           1319
search_and_rescue           725
security                    471
military                    863
child_alone                   0
water                      1684
food                       2956
shelter                    2342
clothing                    410
money                       606
missing_people              301
refugees                    880
death                      1204
other_aid                  3464
infrastructure_related     1709
transport                  1212
buildings                  1345
electricity                 538
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1155
weather_related            7343
floods                     2177
storm                      2464
fire    

In [9]:
for column in Y.columns:
    print(Y[column].value_counts())

1    20042
0     6140
2      204
Name: related, dtype: int64
0    21873
1     4513
Name: request, dtype: int64
0    26265
1      121
Name: offer, dtype: int64
0    15432
1    10954
Name: aid_related, dtype: int64
0    24287
1     2099
Name: medical_help, dtype: int64
0    25067
1     1319
Name: medical_products, dtype: int64
0    25661
1      725
Name: search_and_rescue, dtype: int64
0    25915
1      471
Name: security, dtype: int64
0    25523
1      863
Name: military, dtype: int64
0    26386
Name: child_alone, dtype: int64
0    24702
1     1684
Name: water, dtype: int64
0    23430
1     2956
Name: food, dtype: int64
0    24044
1     2342
Name: shelter, dtype: int64
0    25976
1      410
Name: clothing, dtype: int64
0    25780
1      606
Name: money, dtype: int64
0    26085
1      301
Name: missing_people, dtype: int64
0    25506
1      880
Name: refugees, dtype: int64
0    25182
1     1204
Name: death, dtype: int64
0    22922
1     3464
Name: other_aid, dtype: int64
0    24677
1    

In [10]:
Y.replace(2, 1, inplace=True)

In [11]:
for column in Y.columns:
    print(Y[column].value_counts())

1    20246
0     6140
Name: related, dtype: int64
0    21873
1     4513
Name: request, dtype: int64
0    26265
1      121
Name: offer, dtype: int64
0    15432
1    10954
Name: aid_related, dtype: int64
0    24287
1     2099
Name: medical_help, dtype: int64
0    25067
1     1319
Name: medical_products, dtype: int64
0    25661
1      725
Name: search_and_rescue, dtype: int64
0    25915
1      471
Name: security, dtype: int64
0    25523
1      863
Name: military, dtype: int64
0    26386
Name: child_alone, dtype: int64
0    24702
1     1684
Name: water, dtype: int64
0    23430
1     2956
Name: food, dtype: int64
0    24044
1     2342
Name: shelter, dtype: int64
0    25976
1      410
Name: clothing, dtype: int64
0    25780
1      606
Name: money, dtype: int64
0    26085
1      301
Name: missing_people, dtype: int64
0    25506
1      880
Name: refugees, dtype: int64
0    25182
1     1204
Name: death, dtype: int64
0    22922
1     3464
Name: other_aid, dtype: int64
0    24677
1     1709
Name:

In [12]:
X = df['message']

In [13]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return tokens

In [12]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator = RandomForestClassifier())),
                    ])

In [34]:
from sklearn.model_selection import train_test_split

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

# display results
accuracy = (y_pred == y_test).mean()

In [14]:
category_names = Y.columns

In [14]:
y_test_np = y_test.values

NameError: name 'y_test' is not defined

In [41]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import classification_report

In [42]:
for i, c in enumerate(category_names): 
    print(c) 
    print(classification_report(y_test_np[i], y_pred[i]))

related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       1.00      0.33      0.50         3

    accuracy                           0.94        36
   macro avg       0.97      0.67      0.74        36
weighted avg       0.95      0.94      0.93        36

request
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        31
           1       1.00      0.40      0.57         5

    accuracy                           0.92        36
   macro avg       0.96      0.70      0.76        36
weighted avg       0.92      0.92      0.90        36

offer
              precision    recall  f1-score   support

           0       0.87      0.96      0.92        28
           1       0.80      0.50      0.62         8

    accuracy                           0.86        36
   macro avg       0.84      0.73      0.77        36
weighted avg       0.86      0.86      0.85        3

/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.96        32
           1       1.00      0.25      0.40         4

    accuracy                           0.92        36
   macro avg       0.96      0.62      0.68        36
weighted avg       0.92      0.92      0.89        36

food
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00         1

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

shelter
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        28
           1       1.00      0.50      0.67         8

    accuracy                           0.89        36
   macro avg       0.94      0.75      0.80        36
weighted avg       0.90      0.89      0.87        36

clothi

/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       0.00      0.00      0.00         1

    accuracy                           0.97        36
   macro avg       0.49      0.50      0.49        36
weighted avg       0.95      0.97      0.96        36

storm
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        33
           1       1.00      0.67      0.80         3

    accuracy                           0.97        36
   macro avg       0.99      0.83      0.89        36
weighted avg       0.97      0.97      0.97        36

fire
              precision    recall  f1-score   support

           0       0.86      1.00      0.92        30
           1       1.00      0.17      0.29         6

    accuracy                           0.86        36
   macro avg       0.93      0.58      0.60        36
weighted avg       0.88      0.86      0.82        36

earthqua

In [16]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator = RandomForestClassifier())),
                    ])

from sklearn.model_selection import train_test_split

# perform train test split
X_train_grid, X_test_grid, y_train_grid, y_test_grid = train_test_split(X, Y, test_size=0.2, random_state=42)
 
# Setting up randomforest params
rf_params = {
 'clf__estimator__criterion': ['gini', 'entropy'],
 'clf__estimator__max_depth': [2, 5, None],
 'clf__estimator__n_estimators': [10, 20, 50],
 'clf__estimator__min_samples_leaf':[1, 5, 10],
}

import time
start=time.time()
cv = GridSearchCV(estimator = pipeline, param_grid = rf_params, scoring='f1_macro', cv=2, n_jobs=-1)
cv.fit(X_train_grid, y_train_grid)
y_pred_grid = cv.predict(X_test_grid)


print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  7005.0 seconds


In [17]:
y_test_np_grid = y_test_grid.values

In [19]:
from sklearn.metrics import classification_report
for i, c in enumerate(category_names): 
    print(c) 
    print(classification_report(y_test_np_grid[i], y_pred_grid[i]))

related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       1.00      0.33      0.50         3

    accuracy                           0.94        36
   macro avg       0.97      0.67      0.74        36
weighted avg       0.95      0.94      0.93        36

request
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        31
           1       1.00      0.40      0.57         5

    accuracy                           0.92        36
   macro avg       0.96      0.70      0.76        36
weighted avg       0.92      0.92      0.90        36

offer
              precision    recall  f1-score   support

           0       0.87      0.96      0.92        28
           1       0.80      0.50      0.62         8

    accuracy                           0.86        36
   macro avg       0.84      0.73      0.77        36
weighted avg       0.86      0.86      0.85        3

              precision    recall  f1-score   support

           0       1.00      0.97      0.99        35
           1       0.50      1.00      0.67         1

    accuracy                           0.97        36
   macro avg       0.75      0.99      0.83        36
weighted avg       0.99      0.97      0.98        36



/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__n_estimators': 50}

In [21]:
X_train_grid_2, X_test_grid_2, y_train_grid_2, y_test_grid_2 = train_test_split(X, Y, test_size=0.2, random_state=42)
 
# Setting up randomforest params
rf_params_2 = {
 'clf__estimator__criterion': ['gini'],
 'clf__estimator__max_depth': [None],
 'clf__estimator__n_estimators': [50, 100, 200],
 'clf__estimator__min_samples_leaf':[0.5, 1],
}

import time
start=time.time()
cv_2 = GridSearchCV(estimator = pipeline, param_grid = rf_params_2, scoring='f1_macro', cv=2, n_jobs=-1)
cv_2.fit(X_train_grid_2, y_train_grid_2)
y_pred_grid2 = cv_2.predict(X_test_grid_2)


print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  2148.0 seconds


In [22]:
y_test_np_grid_2 = y_test_grid_2.values

In [23]:
for i, c in enumerate(category_names): 
    print(c) 
    print(classification_report(y_test_np_grid_2[i], y_pred_grid2[i]))

related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       1.00      0.33      0.50         3

    accuracy                           0.94        36
   macro avg       0.97      0.67      0.74        36
weighted avg       0.95      0.94      0.93        36

request
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        31
           1       1.00      0.40      0.57         5

    accuracy                           0.92        36
   macro avg       0.96      0.70      0.76        36
weighted avg       0.92      0.92      0.90        36

offer
              precision    recall  f1-score   support

           0       0.87      0.96      0.92        28
           1       0.80      0.50      0.62         8

    accuracy                           0.86        36
   macro avg       0.84      0.73      0.77        36
weighted avg       0.86      0.86      0.85        3

/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98        31
           1       1.00      0.80      0.89         5

    accuracy                           0.97        36
   macro avg       0.98      0.90      0.94        36
weighted avg       0.97      0.97      0.97        36

other_aid
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.60      0.75         5

    accuracy                           0.94        36
   macro avg       0.97      0.80      0.86        36
weighted avg       0.95      0.94      0.94        36

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        32
           1       1.00      0.50      0.67         4

    accuracy                           0.94        36
   macro avg       0.97      0.75      0.82        36
weighted avg       0.95      0.94      0.

/Users/gabrielpilao1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
cv_2.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__n_estimators': 100}

In [41]:
import pickle
with open('model.pkl', 'wb') as model_file:
    pickle.dump(pipeline, model_file)